In [29]:
%matplotlib inline  
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from keras import metrics
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import NearMiss
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import  multilabel_confusion_matrix

from numpy.random import seed
from tensorflow import set_random_seed
set_random_seed(2)
seed(1)

In [30]:
train_data = pd.read_csv("data/train_data.csv").values
test_data = pd.read_csv("data/test_data.csv").values
train_labels = pd.read_csv("data/train_labels.csv",header=None).values.reshape(-1)
test_labels = pd.read_csv("data/test_labels.csv",header=None).values.reshape(-1)

In [31]:
print(train_data.shape,train_labels.shape,test_data.shape,test_labels.shape)

(65809, 155) (65809,) (43201, 155) (43201,)


In [32]:
print(Counter(train_labels),Counter(test_labels))

Counter({0: 65472, 2: 235, 1: 102}) Counter({0: 42969, 2: 189, 1: 43})


In [33]:

def convert_to_cat(labels):
    enc = LabelEncoder()
    enc_y = enc.fit_transform(labels)
    categorical_y = np_utils.to_categorical(enc_y)
    print(categorical_y.shape,categorical_y[:2])
    return categorical_y
    

In [16]:
# from sklearn.decomposition import PCA
# total_X = np.concatenate([train_data,test_data])

# print("Original Shape ",total_X.shape)
# pca = PCA(n_components=2,svd_solver="full")
# total_X = pca.fit_transform(total_X)
# print("Reduced ",total_X.shape)

# train_x = total_X[:train_data.shape[0]]
# test_x = total_X[train_data.shape[0]:]
# print(train_x.shape,test_x.shape)

In [34]:
from keras.optimizers import SGD, Adadelta, Adam
from keras.losses import mean_squared_error,mean_absolute_error, logcosh, categorical_crossentropy, sparse_categorical_crossentropy
from keras.initializers import RandomNormal, RandomUniform, glorot_normal
from keras.callbacks import TerminateOnNaN, LearningRateScheduler, EarlyStopping

cb1 = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=0, mode='auto', restore_best_weights=False)
cb2 = TerminateOnNaN()
cb = [cb1,cb2]


adam_beta = [[0.9,0.999],[0.8,0.888],[0.7,0.777]]
more_optim = []
lr = [0.1,0.01,0.001]
for l in lr:
    more_optim.append(SGD(lr=l))
    more_optim.append(Adadelta(lr=l))
    for b in adam_beta:
        more_optim.append(Adam(lr=l,beta_1=b[0],beta_2=b[1]))

optim = ["sgd","adadelta","adam"]
loss = ["categorical_crossentropy"] #,"sparse_categorical_crossentropy"]
init = ['glorot_uniform','random_uniform','random_normal']

In [35]:

n_steps = 6
n_features = train_data.shape[1]
model = Sequential()

# Embeddings are not useful in this setting
#model.add(Embedding(124,32,input_length=155))

#model.add(LSTM(units=124,activation='relu',return_sequences=True))
#model.add(LSTM(units=64, activation='relu',return_sequences=True))
model.add(LSTM(units=16,activation='relu',input_shape=(n_steps,n_features)))
model.add(Dense(3, activation='softmax'))



In [36]:
# provide batch data with time_steps for feeding to LSTM 

#y_binary = to_categorical(y_int)
def process_batch(data,lbls,time_steps=3):
    dt = []
    labels = []
    i = time_steps
    while i < len(data):
        dt.append(data[i-time_steps:i])
        labels.append(lbls[i])
        i +=1  
    return np.asarray(dt),np.asarray(labels)


In [39]:
batch_size = 32
n_loss = "categorical_crossentropy"

if n_loss == "categorical_crossentropy":
    categorical_y = convert_to_cat(train_labels)
    train_in,train_lbl = process_batch(train_data,categorical_y,n_steps)
else:
    train_in,train_lbl = process_batch(train_data,train_labels,n_steps)

print(train_in.shape,train_lbl.shape)
n_samples = len(train_in)
d_sample_weight = class_weight.compute_sample_weight('balanced',train_lbl)
print(d_sample_weight)


(65809, 3) [[1. 0. 0.]
 [1. 0. 0.]]
(65803, 6, 155) (65803, 3)
[0.12628954 0.12628954 0.12628954 ... 0.12628954 0.12628954 0.12628954]


In [41]:

# training_generator = BalancedBatchGenerator(train_in,train_lbl, sampler=NearMiss(), batch_size=10, random_state=42)
# history = model.fit_generator(generator=training_generator,epochs=10, verbose=0)

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss=n_loss, optimizer=opt,metrics=["accuracy","categorical_accuracy"])
history = model.fit(train_in,train_lbl,epochs=10,validation_split=0.2,sample_weight=d_sample_weight,batch_size=batch_size,callbacks=cb)

## sparse_categoricalcrossentropy expects lebel encoded target whereas categoricalcrossentropy OHE
# for opt in more_optim:
#     print(" OPT ",t)
#     for ls in loss:
#         print(opt,ls,"------------------------------")
#         model.compile(loss=ls, optimizer=opt,metrics=["accuracy","categorical_accuracy"])
#         history = model.fit(train_in,train_lbl,epochs=20,validation_split=0.2,sample_weight=d_sample_weight,batch_size=batch_size,callbacks=cb)
#     t+=1

Train on 52642 samples, validate on 13161 samples
Epoch 1/20
52642/52642 [==============================] - 23s 428us/step - loss: 606.6251 - acc: 0.0031 - categorical_accuracy: 0.0031 - val_loss: 2.0323 - val_acc: 0.0016 - val_categorical_accuracy: 0.0016
Epoch 2/20
52642/52642 [==============================] - 16s 313us/step - loss: 606.6251 - acc: 0.0031 - categorical_accuracy: 0.0031 - val_loss: 2.0323 - val_acc: 0.0016 - val_categorical_accuracy: 0.0016
Epoch 3/20
52642/52642 [==============================] - 11s 208us/step - loss: 606.6251 - acc: 0.0031 - categorical_accuracy: 0.0031 - val_loss: 2.0323 - val_acc: 0.0016 - val_categorical_accuracy: 0.0016
Epoch 4/20
12160/52642 [=====>........................] - ETA: 10s - loss: 604.5371 - acc: 0.0029 - categorical_accuracy: 0.0029

KeyboardInterrupt: 

In [23]:
from keras.models import load_model

model.save('complete_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model


# model = load_model('complete_model.h5')

categorical_y = convert_to_cat(test_labels)
test_in,test_lbl = process_batch(test_data,categorical_y,time_steps)

results = model.evaluate(test_in, test_lbl)
predictions = model.predict(test_in)
print("Accuracy: %.2f%%" % (results[1]*100))


(43201, 3) [[1. 0. 0.]
 [1. 0. 0.]]
43191/43191 [==============================] - 7s 155us/step
Accuracy: 99.34%


In [24]:
y_pred = predictions.argmax(1)
print(predictions[:2])
print(test_lbl[:2])

[[1. 0. 0.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]]


In [28]:
lbl = test_lbl.argmax(1)
print(confusion_matrix(lbl,y_pred))


[[42907    52     0]
 [   43     0     0]
 [  189     0     0]]


In [ ]:
acc = history.history['acc']
#val_acc = history.history['val_acc']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
#plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()